Loading required packages

In [2]:
import pandas as pd
import boto3
from datetime import datetime
import plotly.graph_objects as go

Reading in the data from the S3 bucket (don't forget to pip install boto3)

In [2]:
# meteo data
Q1_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q1.csv')
Q2_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q2.csv')
Q3_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q3.csv')
Q4_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q4.csv')

In [ ]:
# REMARK: this is the 'old' noise data, don't run this

# noise data
exp40_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
exp40_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
exp40_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
exp40_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
exp40_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
exp40_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
exp40_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
exp40_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_280324_mp08bis---vrijthof.csv', header=0, sep=';')
exp40_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_303910_mp-04-his-hears.csv', header=0, sep=';')

exp41_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
exp41_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
exp41_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
exp41_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
exp41_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
exp41_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
exp41_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
exp41_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_280324_mp08bis---vrijthof.csv', header=0, sep=';')
exp41_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_303910_mp-04-his-hears.csv', header=0, sep=';')

exp42_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
exp42_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
exp42_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
exp42_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
exp42_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
exp42_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
exp42_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
exp42_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
exp42_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')


In [3]:
# updated noise data - January
naamse35_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')

In [4]:
# updated noise data - February
naamse35_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')

In [20]:
# updated noise data - March
naamse35_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_303910_mp-04-his-hears.csv', header=0, sep=';')

In [21]:
# updated noise data - April
naamse35_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# updated noise data - May
naamse35_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# updated noise data - June
naamse35_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_303910_mp-04-his-hears.csv', header=0, sep=';')

In [3]:
# updated noise data - July
naamse35_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_303910_mp-04-his-hears.csv', header=0, sep=';')

In [4]:
# updated noise data - August
naamse35_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_303910_mp-04-his-hears.csv', header=0, sep=';')

In [9]:
# updated noise data - September
naamse35_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_303910_mp-04-his-hears.csv', header=0, sep=';')

In [10]:
# updated noise data - October
naamse35_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_303910_mp-04-his-hears.csv', header=0, sep=';')

In [11]:
# updated noise data - November
naamse35_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_303910_mp-04-his-hears.csv', header=0, sep=';')

In [3]:
# updated noise data - December
naamse35_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
naamse57_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
naamse62_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
calvarie_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
park_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
naamse81_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
kiosk_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
vrijt_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_280324_mp08bis---vrijthof.csv', header=0, sep=';')
his_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_303910_mp-04-his-hears.csv', header=0, sep=';')

### Combining and aggregating the meteo data

In [5]:
# combine meteo dataset 
meteocombined = pd.concat([Q1_2022, Q2_2022, Q3_2022, Q4_2022], axis=0)
meteocombined.head()

,DATEUTC,ID,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,...,Year,Month,Day,Hour,Minute,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022-01-01 00:10:00,LC-002,92.0,11.78,38.0,0.0,0.0,0.0,-169.0,0.43,...,2022,1,1,0,10,0.0,13.11,13.11,13.0515,13.048027
1,2022-01-01 00:20:00,LC-002,92.0,11.73,37.0,0.0,0.0,0.0,-170.0,0.33,...,2022,1,1,0,20,0.0,13.01,13.01,12.9515,12.985849
2,2022-01-01 00:30:00,LC-002,92.0,11.73,38.0,0.0,0.0,0.0,-167.0,0.46,...,2022,1,1,0,30,0.0,13.00,13.00,12.9415,12.950322
3,2022-01-01 00:40:00,LC-002,92.0,11.72,37.0,0.0,0.0,0.0,-160.0,0.52,...,2022,1,1,0,40,0.0,13.00,13.00,12.9415,12.949550
4,2022-01-01 00:50:00,LC-002,92.0,11.72,38.0,0.0,0.0,0.0,-166.0,0.51,...,2022,1,1,0,50,0.0,13.00,13.00,12.9415,12.952268


In [6]:
# check for missing values in each column
print(meteocombined.isnull().sum())

DATEUTC              0
ID                   0
LC_HUMIDITY     314899
LC_DWPTEMP      314899
LC_n            314899
LC_RAD          314899
LC_RAININ       314899
LC_DAILYRAIN    314899
LC_WINDDIR      314899
LC_WINDSPEED    314899
Date                 0
Year                 0
Month                0
Day                  0
Hour                 0
Minute               0
LC_RAD60        277022
LC_TEMP_QCL0    314899
LC_TEMP_QCL1    345487
LC_TEMP_QCL2    345487
LC_TEMP_QCL3    345487
dtype: int64


In [8]:
# aggregate meteo data by day
avg_meteo_combined = meteocombined.groupby(['Year','Month', 'Day']).mean()
avg_meteo_combined = avg_meteo_combined.reset_index()
avg_meteo_combined.head()


,Year,Month,Day,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Hour,Minute,LC_RAD60,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3
0,2022,1,1,86.557205,10.257703,36.346609,21.264531,0.000002,0.000029,-6.567639,0.322477,11.58042,25.174825,21.396634,12.447359,12.464281,12.450807,12.424740
1,2022,1,2,83.512460,9.379296,36.056435,14.140794,0.000658,0.003468,-28.866315,0.721487,11.50000,25.000000,14.099483,12.112883,12.115685,12.102359,12.086891
2,2022,1,3,89.579036,8.004425,35.882086,5.529369,0.000750,0.006831,-35.256016,0.663436,11.50000,25.000000,5.517430,9.646861,9.648249,9.635166,9.683051
3,2022,1,4,94.288450,5.983424,35.816041,7.944379,0.000469,0.003571,-25.637291,0.333304,11.50000,25.000000,7.962571,6.830167,6.828260,6.816515,6.837235
4,2022,1,5,88.026102,2.078484,35.573298,13.893690,0.000131,0.000262,-44.342911,0.609569,11.50000,25.000000,13.890103,3.903085,3.900556,3.890475,3.915760


In [9]:
month_max_value = avg_meteo_combined['Month'].max()
print(f"This combined meteo dataset contains the weather data for all {month_max_value} months.")

This combined meteo dataset contains the weather data for all 12 months.


### Combining and aggregating the noise data

- January

In [12]:
# combine noise data for January together
noise_jan_combined = pd.concat([naamse35_jan, naamse57_jan, naamse62_jan, calvarie_jan, park_jan, naamse81_jan, kiosk_jan, vrijt_jan, his_jan], axis=0)
noise_jan_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,dB(C)
1,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,dB(C)
2,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,dB(C)
3,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,dB(C)
4,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,dB(C)


In [13]:
# extract the date, month, hour, minute of "result_timestamp"
noise_jan_combined['result_timestamp'] = pd.to_datetime(noise_jan_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_jan_combined['result_date'] = noise_jan_combined['result_timestamp'].dt.date
noise_jan_combined['result_month'] = noise_jan_combined['result_timestamp'].dt.month
noise_jan_combined['result_day'] = noise_jan_combined['result_timestamp'].dt.day
noise_jan_combined['result_hour'] = noise_jan_combined['result_timestamp'].dt.hour
noise_jan_combined['result_minute'] = noise_jan_combined['result_timestamp'].dt.minute

noise_jan_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,dB(C),2022-01-01,1,1,0,0
1,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,dB(C),2022-01-01,1,1,0,0
2,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,dB(C),2022-01-01,1,1,0,0
3,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,dB(C),2022-01-01,1,1,0,0
4,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,dB(C),2022-01-01,1,1,0,0


In [14]:
# aggregate the data by day
avg_jan_combined = noise_jan_combined.groupby(['result_date','description']).mean()
avg_jan_combined = avg_jan_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_jan_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_jan_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-01-01,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,59.345285,71.190601,1.0,1.0
1,2022-01-01,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,54.970850,67.557919,1.0,1.0
2,2022-01-01,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,54.286131,67.321329,1.0,1.0
3,2022-01-01,MP 07: Naamsestraat 81,45.908501,44.373056,55.526650,66.988945,1.0,1.0
4,2022-01-02,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,59.816366,71.545452,1.0,2.0
...,...,...,...,...,...,...,...,...
119,2022-01-30,MP 07: Naamsestraat 81,45.735373,44.415840,56.260835,67.496251,1.0,30.0
120,2022-01-31,MP 03: Naamsestraat 62 Taste,54.224553,52.693739,63.503406,75.360208,1.0,31.0
121,2022-01-31,MP 05: Calvariekapel KU Leuven,52.480993,50.720293,60.623738,72.883390,1.0,31.0
122,2022-01-31,MP 06: Parkstraat 2 La Filosovia,51.580934,49.908136,58.852705,71.408037,1.0,31.0


In [15]:
# check for missing values in each column
print(avg_jan_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- February

In [16]:
# combine noise data for February together
noise_feb_combined = pd.concat([naamse35_feb, naamse57_feb, naamse62_feb, calvarie_feb, park_feb, naamse81_feb, kiosk_feb, vrijt_feb, his_feb], axis=0)
noise_feb_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,28/02/2022 08:23:26.734,72.5,dB(A),71.8,dB(A),80.80,dB(C),92.17,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,28/02/2022 08:23:27.734,71.1,dB(A),68.8,dB(A),78.96,dB(C),90.57,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,28/02/2022 08:23:28.734,72.0,dB(A),70.3,dB(A),78.64,dB(C),91.05,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,28/02/2022 08:23:29.734,69.2,dB(A),68.5,dB(A),76.94,dB(C),88.43,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,28/02/2022 08:23:30.734,70.6,dB(A),69.7,dB(A),76.75,dB(C),90.19,dB(C)


In [17]:
# extract the date, month, hour, minute of "result_timestamp"
noise_feb_combined['result_timestamp'] = pd.to_datetime(noise_feb_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_feb_combined['result_date'] = noise_feb_combined['result_timestamp'].dt.date
noise_feb_combined['result_month'] = noise_feb_combined['result_timestamp'].dt.month
noise_feb_combined['result_day'] = noise_feb_combined['result_timestamp'].dt.day
noise_feb_combined['result_hour'] = noise_feb_combined['result_timestamp'].dt.hour
noise_feb_combined['result_minute'] = noise_feb_combined['result_timestamp'].dt.minute

noise_feb_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:26.734,72.5,dB(A),71.8,dB(A),80.80,dB(C),92.17,dB(C),2022-02-28,2,28,8,23
1,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:27.734,71.1,dB(A),68.8,dB(A),78.96,dB(C),90.57,dB(C),2022-02-28,2,28,8,23
2,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:28.734,72.0,dB(A),70.3,dB(A),78.64,dB(C),91.05,dB(C),2022-02-28,2,28,8,23
3,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:29.734,69.2,dB(A),68.5,dB(A),76.94,dB(C),88.43,dB(C),2022-02-28,2,28,8,23
4,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:30.734,70.6,dB(A),69.7,dB(A),76.75,dB(C),90.19,dB(C),2022-02-28,2,28,8,23


In [18]:
# aggregate the data by day
avg_feb_combined = noise_feb_combined.groupby(['result_date','description']).mean()
avg_feb_combined = avg_feb_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_feb_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_feb_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-02-01,MP 03: Naamsestraat 62 Taste,54.948997,53.332087,63.025565,74.671864,2.0,1.0
1,2022-02-01,MP 05: Calvariekapel KU Leuven,53.203722,51.239140,59.742471,72.082363,2.0,1.0
2,2022-02-01,MP 06: Parkstraat 2 La Filosovia,53.183853,51.231713,59.521015,72.243594,2.0,1.0
3,2022-02-01,MP 07: Naamsestraat 81,51.413080,50.029712,60.851355,72.261557,2.0,1.0
4,2022-02-02,MP 03: Naamsestraat 62 Taste,54.271621,52.578714,62.396267,74.063853,2.0,2.0
...,...,...,...,...,...,...,...,...
109,2022-02-28,MP 02: Naamsestraat 57 Xior,52.207648,50.042913,57.411352,70.014035,2.0,28.0
110,2022-02-28,MP 03: Naamsestraat 62 Taste,54.236143,52.443441,62.302081,73.940075,2.0,28.0
111,2022-02-28,MP 05: Calvariekapel KU Leuven,52.368430,50.206850,58.683154,71.086718,2.0,28.0
112,2022-02-28,MP 06: Parkstraat 2 La Filosovia,52.073699,50.110397,58.661772,71.234600,2.0,28.0


In [19]:
# check for missing values in each column
print(avg_feb_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- March

In [ ]:
# combine noise data for March together
noise_mar_combined = pd.concat([naamse35_mar, naamse57_mar, naamse62_mar, calvarie_mar, park_mar, naamse81_mar, kiosk_mar, vrijt_mar, his_mar], axis=0)
noise_mar_combined.head()

In [ ]:
# extract the date, month, hour, minute of "result_timestamp"
noise_mar_combined['result_timestamp'] = pd.to_datetime(noise_mar_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_mar_combined['result_date'] = noise_mar_combined['result_timestamp'].dt.date
noise_mar_combined['result_month'] = noise_mar_combined['result_timestamp'].dt.month
noise_mar_combined['result_day'] = noise_mar_combined['result_timestamp'].dt.day
noise_mar_combined['result_hour'] = noise_mar_combined['result_timestamp'].dt.hour
noise_mar_combined['result_minute'] = noise_mar_combined['result_timestamp'].dt.minute

noise_mar_combined.head()

In [ ]:
# aggregate the data by day
avg_mar_combined = noise_mar_combined.groupby(['result_date','description']).mean()
avg_mar_combined = avg_mar_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_mar_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_mar_combined.head(150)

In [ ]:
# check for missing values in each column
print(avg_mar_combined.isnull().sum())

- April 

In [ ]:
# combine noise data for April together
noise_apr_combined = pd.concat([naamse35_apr, naamse57_apr, naamse62_apr, calvarie_apr, park_apr, naamse81_apr, kiosk_apr, vrijt_apr, his_apr], axis=0)
noise_apr_combined.head()

In [ ]:
# extract the date, month, hour, minute of "result_timestamp"
noise_apr_combined['result_timestamp'] = pd.to_datetime(noise_apr_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_apr_combined['result_date'] = noise_apr_combined['result_timestamp'].dt.date
noise_apr_combined['result_month'] = noise_apr_combined['result_timestamp'].dt.month
noise_apr_combined['result_day'] = noise_apr_combined['result_timestamp'].dt.day
noise_apr_combined['result_hour'] = noise_apr_combined['result_timestamp'].dt.hour
noise_apr_combined['result_minute'] = noise_apr_combined['result_timestamp'].dt.minute

noise_apr_combined.head()

In [ ]:
# aggregate the data by day
avg_apr_combined = noise_apr_combined.groupby(['result_date','description']).mean()
avg_apr_combined = avg_apr_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_apr_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_apr_combined.head(150)

In [ ]:
# check for missing values in each column
print(avg_apr_combined.isnull().sum())

- May 

In [ ]:
# combine noise data for May together
noise_may_combined = pd.concat([naamse35_may, naamse57_may, naamse62_may, calvarie_may, park_may, naamse81_may, kiosk_may, vrijt_may, his_may], axis=0)
noise_may_combined.head()

In [ ]:
# extract the date, month, hour, minute of "result_timestamp"
noise_may_combined['result_timestamp'] = pd.to_datetime(noise_may_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_may_combined['result_date'] = noise_may_combined['result_timestamp'].dt.date
noise_may_combined['result_month'] = noise_may_combined['result_timestamp'].dt.month
noise_may_combined['result_day'] = noise_may_combined['result_timestamp'].dt.day
noise_may_combined['result_hour'] = noise_may_combined['result_timestamp'].dt.hour
noise_may_combined['result_minute'] = noise_may_combined['result_timestamp'].dt.minute

noise_may_combined.head()

In [ ]:
# aggregate the data by day
avg_may_combined = noise_may_combined.groupby(['result_date','description']).mean()
avg_may_combined = avg_may_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_may_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_may_combined.head(150)

In [ ]:
# check for missing values in each column
print(avg_may_combined.isnull().sum())

- June

In [ ]:
# combine noise data for June together
noise_jun_combined = pd.concat([naamse35_jun, naamse57_jun, naamse62_jun, calvarie_jun, park_jun, naamse81_jun, kiosk_jun, vrijt_jun, his_jun], axis=0)
noise_jun_combined.head()

In [ ]:
# extract the date, month, hour, minute of "result_timestamp"
noise_jun_combined['result_timestamp'] = pd.to_datetime(noise_jun_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_jun_combined['result_date'] = noise_jun_combined['result_timestamp'].dt.date
noise_jun_combined['result_month'] = noise_jun_combined['result_timestamp'].dt.month
noise_jun_combined['result_day'] = noise_jun_combined['result_timestamp'].dt.day
noise_jun_combined['result_hour'] = noise_jun_combined['result_timestamp'].dt.hour
noise_jun_combined['result_minute'] = noise_jun_combined['result_timestamp'].dt.minute

noise_jun_combined.head()

In [ ]:
# aggregate the data by day
avg_jun_combined = noise_jun_combined.groupby(['result_date','description']).mean()
avg_jun_combined = avg_jun_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_jun_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_jun_combined.head(150)

In [ ]:
# check for missing values in each column
print(avg_jun_combined.isnull().sum())

- July

In [5]:
# combine noise data for July together
noise_jul_combined = pd.concat([naamse35_jul, naamse57_jul, naamse62_jul, calvarie_jul, park_jul, naamse81_jul, kiosk_jul, vrijt_jul, his_jul], axis=0)
noise_jul_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,01/07/2022 00:00:00.156,53.6,dB(A),52.9,dB(A),62.59,dB(C),74.99,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,01/07/2022 00:00:01.156,54.2,dB(A),53.0,dB(A),62.59,dB(C),74.55,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,01/07/2022 00:00:02.156,55.2,dB(A),54.1,dB(A),63.10,dB(C),75.95,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,01/07/2022 00:00:03.156,55.8,dB(A),55.1,dB(A),63.55,dB(C),73.95,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,01/07/2022 00:00:04.156,55.7,dB(A),54.8,dB(A),64.27,dB(C),75.74,dB(C)


In [6]:
# extract the date, month, hour, minute of "result_timestamp"
noise_jul_combined['result_timestamp'] = pd.to_datetime(noise_jul_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_jul_combined['result_date'] = noise_jul_combined['result_timestamp'].dt.date
noise_jul_combined['result_month'] = noise_jul_combined['result_timestamp'].dt.month
noise_jul_combined['result_day'] = noise_jul_combined['result_timestamp'].dt.day
noise_jul_combined['result_hour'] = noise_jul_combined['result_timestamp'].dt.hour
noise_jul_combined['result_minute'] = noise_jul_combined['result_timestamp'].dt.minute

noise_jul_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-07-01 00:00:00.156,53.6,dB(A),52.9,dB(A),62.59,dB(C),74.99,dB(C),2022-07-01,7,1,0,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-07-01 00:00:01.156,54.2,dB(A),53.0,dB(A),62.59,dB(C),74.55,dB(C),2022-07-01,7,1,0,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-07-01 00:00:02.156,55.2,dB(A),54.1,dB(A),63.10,dB(C),75.95,dB(C),2022-07-01,7,1,0,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-07-01 00:00:03.156,55.8,dB(A),55.1,dB(A),63.55,dB(C),73.95,dB(C),2022-07-01,7,1,0,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-07-01 00:00:04.156,55.7,dB(A),54.8,dB(A),64.27,dB(C),75.74,dB(C),2022-07-01,7,1,0,0


In [7]:
# aggregate the data by day
avg_jul_combined = noise_jul_combined.groupby(['result_date','description']).mean()
avg_jul_combined = avg_jul_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_jul_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_jul_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-07-01,MP 01: Naamsestraat 35 Maxim,55.299206,53.146303,63.415607,75.369154,7.0,1.0
1,2022-07-01,MP 02: Naamsestraat 57 Xior,54.593376,52.582111,62.253037,74.105457,7.0,1.0
2,2022-07-01,MP 03: Naamsestraat 62 Taste,55.377303,53.543788,63.005621,74.642645,7.0,1.0
3,2022-07-01,MP 05: Calvariekapel KU Leuven,54.025243,51.824704,59.799215,72.117770,7.0,1.0
4,2022-07-01,MP 06: Parkstraat 2 La Filosovia,53.680466,51.675904,60.049342,72.131190,7.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-07-21,MP 07: Naamsestraat 81,46.636930,45.012391,54.354061,66.226709,7.0,21.0
146,2022-07-21,MP08bis - Vrijthof,46.287975,44.775370,55.763282,67.631967,7.0,21.0
147,2022-07-22,MP 01: Naamsestraat 35 Maxim,52.836045,50.412693,62.963130,75.278902,7.0,22.0
148,2022-07-22,MP 02: Naamsestraat 57 Xior,52.815758,50.520978,60.821460,73.153473,7.0,22.0


In [8]:
# check for missing values in each column
print(avg_jul_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- August

In [12]:
# combine noise data for August together
noise_aug_combined = pd.concat([naamse35_aug, naamse57_aug, naamse62_aug, calvarie_aug, park_aug, naamse81_aug, kiosk_aug, vrijt_aug, his_aug], axis=0)
noise_aug_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,01/08/2022 00:00:00.973,59.2,dB(A),57.4,dB(A),66.15,dB(C),77.82,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,01/08/2022 00:00:01.973,57.6,dB(A),56.9,dB(A),66.99,dB(C),77.41,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,01/08/2022 00:00:02.973,59.4,dB(A),58.1,dB(A),66.22,dB(C),76.82,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,01/08/2022 00:00:03.973,60.6,dB(A),58.4,dB(A),67.43,dB(C),77.97,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,01/08/2022 00:00:04.973,60.6,dB(A),58.6,dB(A),67.84,dB(C),79.56,dB(C)


In [13]:
# extract the date, month, hour, minute of "result_timestamp"
noise_aug_combined['result_timestamp'] = pd.to_datetime(noise_aug_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_aug_combined['result_date'] = noise_aug_combined['result_timestamp'].dt.date
noise_aug_combined['result_month'] = noise_aug_combined['result_timestamp'].dt.month
noise_aug_combined['result_day'] = noise_aug_combined['result_timestamp'].dt.day
noise_aug_combined['result_hour'] = noise_aug_combined['result_timestamp'].dt.hour
noise_aug_combined['result_minute'] = noise_aug_combined['result_timestamp'].dt.minute

noise_aug_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-08-01 00:00:00.973,59.2,dB(A),57.4,dB(A),66.15,dB(C),77.82,dB(C),2022-08-01,8,1,0,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-08-01 00:00:01.973,57.6,dB(A),56.9,dB(A),66.99,dB(C),77.41,dB(C),2022-08-01,8,1,0,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-08-01 00:00:02.973,59.4,dB(A),58.1,dB(A),66.22,dB(C),76.82,dB(C),2022-08-01,8,1,0,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-08-01 00:00:03.973,60.6,dB(A),58.4,dB(A),67.43,dB(C),77.97,dB(C),2022-08-01,8,1,0,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-08-01 00:00:04.973,60.6,dB(A),58.6,dB(A),67.84,dB(C),79.56,dB(C),2022-08-01,8,1,0,0


In [14]:
# aggregate the data by day
avg_aug_combined = noise_aug_combined.groupby(['result_date','description']).mean()
avg_aug_combined = avg_aug_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_aug_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_aug_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-08-01,MP 01: Naamsestraat 35 Maxim,51.351982,49.129856,60.223307,72.522238,8.0,1.0
1,2022-08-01,MP 02: Naamsestraat 57 Xior,51.204333,49.184507,58.817776,71.125831,8.0,1.0
2,2022-08-01,MP 03: Naamsestraat 62 Taste,53.077894,51.508258,61.263347,72.839668,8.0,1.0
3,2022-08-01,MP 05: Calvariekapel KU Leuven,50.264108,48.275762,57.128319,69.283939,8.0,1.0
4,2022-08-01,MP 06: Parkstraat 2 La Filosovia,49.646613,47.767204,56.836655,69.141624,8.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-08-21,MP 07: Naamsestraat 81,46.630420,45.150669,55.622435,67.019619,8.0,21.0
146,2022-08-21,MP08bis - Vrijthof,44.067094,42.594237,53.692176,65.309689,8.0,21.0
147,2022-08-22,MP 01: Naamsestraat 35 Maxim,52.323612,49.933648,60.478416,73.329171,8.0,22.0
148,2022-08-22,MP 02: Naamsestraat 57 Xior,51.898864,49.929591,59.436122,71.771534,8.0,22.0


In [15]:
# check for missing values in each column
print(avg_aug_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- September

In [16]:
# combine noise data for September together
noise_sep_combined = pd.concat([naamse35_sep, naamse57_sep, naamse62_sep, calvarie_sep, park_sep, naamse81_sep, kiosk_sep, vrijt_sep, his_sep], axis=0)
noise_sep_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,01/09/2022 00:00:00.656,47.5,dB(A),46.1,dB(A),58.31,dB(C),69.33,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,01/09/2022 00:00:01.656,51.0,dB(A),48.4,dB(A),60.05,dB(C),71.59,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,01/09/2022 00:00:02.656,49.8,dB(A),47.5,dB(A),58.15,dB(C),69.80,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,01/09/2022 00:00:03.656,47.9,dB(A),46.1,dB(A),59.10,dB(C),70.95,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,01/09/2022 00:00:04.656,49.9,dB(A),48.7,dB(A),59.16,dB(C),70.35,dB(C)


In [17]:
# extract the date, month, hour, minute of "result_timestamp"
noise_sep_combined['result_timestamp'] = pd.to_datetime(noise_sep_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_sep_combined['result_date'] = noise_sep_combined['result_timestamp'].dt.date
noise_sep_combined['result_month'] = noise_sep_combined['result_timestamp'].dt.month
noise_sep_combined['result_day'] = noise_sep_combined['result_timestamp'].dt.day
noise_sep_combined['result_hour'] = noise_sep_combined['result_timestamp'].dt.hour
noise_sep_combined['result_minute'] = noise_sep_combined['result_timestamp'].dt.minute

noise_sep_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-09-01 00:00:00.656,47.5,dB(A),46.1,dB(A),58.31,dB(C),69.33,dB(C),2022-09-01,9,1,0,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-09-01 00:00:01.656,51.0,dB(A),48.4,dB(A),60.05,dB(C),71.59,dB(C),2022-09-01,9,1,0,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-09-01 00:00:02.656,49.8,dB(A),47.5,dB(A),58.15,dB(C),69.80,dB(C),2022-09-01,9,1,0,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-09-01 00:00:03.656,47.9,dB(A),46.1,dB(A),59.10,dB(C),70.95,dB(C),2022-09-01,9,1,0,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-09-01 00:00:04.656,49.9,dB(A),48.7,dB(A),59.16,dB(C),70.35,dB(C),2022-09-01,9,1,0,0


In [18]:
# aggregate the data by day
avg_sep_combined = noise_sep_combined.groupby(['result_date','description']).mean()
avg_sep_combined = avg_sep_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_sep_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_sep_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-09-01,MP 01: Naamsestraat 35 Maxim,53.621941,51.349821,62.289483,74.537035,9.0,1.0
1,2022-09-01,MP 02: Naamsestraat 57 Xior,52.599292,50.730064,60.937985,72.755168,9.0,1.0
2,2022-09-01,MP 03: Naamsestraat 62 Taste,55.440355,53.622194,63.201378,74.967795,9.0,1.0
3,2022-09-01,MP 05: Calvariekapel KU Leuven,53.245334,51.245556,60.452137,72.408840,9.0,1.0
4,2022-09-01,MP 06: Parkstraat 2 La Filosovia,53.747064,51.847856,61.025457,73.143248,9.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-09-25,MP 02: Naamsestraat 57 Xior,52.182649,50.137119,58.831301,71.340799,9.0,25.0
146,2022-09-25,MP 03: Naamsestraat 62 Taste,53.967483,52.284094,61.270272,73.083477,9.0,25.0
147,2022-09-25,MP 05: Calvariekapel KU Leuven,51.745141,49.511949,57.329064,69.893310,9.0,25.0
148,2022-09-25,MP 06: Parkstraat 2 La Filosovia,51.016977,49.084723,57.228606,69.806156,9.0,25.0


In [19]:
# check for missing values in each column
print(avg_sep_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- October

In [20]:
# combine noise data for Octber together
noise_oct_combined = pd.concat([naamse35_oct, naamse57_oct, naamse62_oct, calvarie_oct, park_oct, naamse81_oct, kiosk_oct, vrijt_oct, his_oct], axis=0)
noise_oct_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,01/10/2022 00:00:00.575,59.3,dB(A),54.3,dB(A),62.27,dB(C),75.57,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,01/10/2022 00:00:01.575,60.3,dB(A),56.1,dB(A),62.02,dB(C),74.51,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,01/10/2022 00:00:02.575,56.8,dB(A),52.5,dB(A),59.83,dB(C),71.57,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,01/10/2022 00:00:03.575,58.5,dB(A),54.1,dB(A),60.62,dB(C),73.94,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,01/10/2022 00:00:04.575,56.0,dB(A),52.6,dB(A),60.97,dB(C),74.38,dB(C)


In [21]:
# extract the date, month, hour, minute of "result_timestamp"
noise_oct_combined['result_timestamp'] = pd.to_datetime(noise_oct_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_oct_combined['result_date'] = noise_oct_combined['result_timestamp'].dt.date
noise_oct_combined['result_month'] = noise_oct_combined['result_timestamp'].dt.month
noise_oct_combined['result_day'] = noise_oct_combined['result_timestamp'].dt.day
noise_oct_combined['result_hour'] = noise_oct_combined['result_timestamp'].dt.hour
noise_oct_combined['result_minute'] = noise_oct_combined['result_timestamp'].dt.minute

noise_oct_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-10-01 00:00:00.575,59.3,dB(A),54.3,dB(A),62.27,dB(C),75.57,dB(C),2022-10-01,10,1,0,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-10-01 00:00:01.575,60.3,dB(A),56.1,dB(A),62.02,dB(C),74.51,dB(C),2022-10-01,10,1,0,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-10-01 00:00:02.575,56.8,dB(A),52.5,dB(A),59.83,dB(C),71.57,dB(C),2022-10-01,10,1,0,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-10-01 00:00:03.575,58.5,dB(A),54.1,dB(A),60.62,dB(C),73.94,dB(C),2022-10-01,10,1,0,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-10-01 00:00:04.575,56.0,dB(A),52.6,dB(A),60.97,dB(C),74.38,dB(C),2022-10-01,10,1,0,0


In [22]:
# aggregate the data by day
avg_oct_combined = noise_oct_combined.groupby(['result_date','description']).mean()
avg_oct_combined = avg_oct_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_oct_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_oct_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-10-01,MP 01: Naamsestraat 35 Maxim,54.190719,51.965542,61.816454,74.322103,10.0,1.0
1,2022-10-01,MP 02: Naamsestraat 57 Xior,54.161268,52.139308,61.898844,74.339812,10.0,1.0
2,2022-10-01,MP 03: Naamsestraat 62 Taste,55.052215,53.353789,61.705684,74.107765,10.0,1.0
3,2022-10-01,MP 05: Calvariekapel KU Leuven,53.850339,51.764201,59.180158,71.806426,10.0,1.0
4,2022-10-01,MP 06: Parkstraat 2 La Filosovia,53.022391,51.202906,58.579117,71.151283,10.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-10-23,MP 05: Calvariekapel KU Leuven,51.255664,49.100032,57.329258,69.727292,10.0,23.0
146,2022-10-23,MP 06: Parkstraat 2 La Filosovia,50.328906,48.460629,56.609714,68.922152,10.0,23.0
147,2022-10-23,MP 07: Naamsestraat 81,48.769109,47.263611,57.259926,68.770061,10.0,23.0
148,2022-10-23,MP08bis - Vrijthof,48.005393,46.830043,57.397100,69.391877,10.0,23.0


In [23]:
# check for missing values in each column
print(avg_oct_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- November

In [24]:
# combine noise data for November together
noise_nov_combined = pd.concat([naamse35_nov, naamse57_nov, naamse62_nov, calvarie_nov, park_nov, naamse81_nov, kiosk_nov, vrijt_nov, his_nov], axis=0)
noise_nov_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,02/11/2022 09:44:56.835,62.4,dB(A),61.6,dB(A),75.09,dB(C),85.14,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,02/11/2022 09:44:57.835,63.3,dB(A),63.0,dB(A),76.14,dB(C),86.22,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,02/11/2022 09:44:58.835,64.1,dB(A),63.7,dB(A),77.72,dB(C),87.75,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,02/11/2022 09:44:59.835,64.1,dB(A),62.0,dB(A),74.61,dB(C),86.46,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,02/11/2022 09:45:00.835,61.8,dB(A),61.2,dB(A),73.98,dB(C),85.30,dB(C)


In [25]:
# extract the date, month, hour, minute of "result_timestamp"
noise_nov_combined['result_timestamp'] = pd.to_datetime(noise_nov_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_nov_combined['result_date'] = noise_nov_combined['result_timestamp'].dt.date
noise_nov_combined['result_month'] = noise_nov_combined['result_timestamp'].dt.month
noise_nov_combined['result_day'] = noise_nov_combined['result_timestamp'].dt.day
noise_nov_combined['result_hour'] = noise_nov_combined['result_timestamp'].dt.hour
noise_nov_combined['result_minute'] = noise_nov_combined['result_timestamp'].dt.minute

noise_nov_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-11-02 09:44:56.835,62.4,dB(A),61.6,dB(A),75.09,dB(C),85.14,dB(C),2022-11-02,11,2,9,44
1,255439,MP 01: Naamsestraat 35 Maxim,2022-11-02 09:44:57.835,63.3,dB(A),63.0,dB(A),76.14,dB(C),86.22,dB(C),2022-11-02,11,2,9,44
2,255439,MP 01: Naamsestraat 35 Maxim,2022-11-02 09:44:58.835,64.1,dB(A),63.7,dB(A),77.72,dB(C),87.75,dB(C),2022-11-02,11,2,9,44
3,255439,MP 01: Naamsestraat 35 Maxim,2022-11-02 09:44:59.835,64.1,dB(A),62.0,dB(A),74.61,dB(C),86.46,dB(C),2022-11-02,11,2,9,44
4,255439,MP 01: Naamsestraat 35 Maxim,2022-11-02 09:45:00.835,61.8,dB(A),61.2,dB(A),73.98,dB(C),85.30,dB(C),2022-11-02,11,2,9,45


In [26]:
# aggregate the data by day
avg_nov_combined = noise_nov_combined.groupby(['result_date','description']).mean()
avg_nov_combined = avg_nov_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_nov_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_nov_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-11-01,MP 03: Naamsestraat 62 Taste,55.014634,53.019300,61.582628,73.697453,11.0,1.0
1,2022-11-01,MP 04: His & Hears,56.653023,54.833671,62.886829,75.056101,11.0,1.0
2,2022-11-01,MP 05: Calvariekapel KU Leuven,53.512625,51.052653,59.115961,71.906545,11.0,1.0
3,2022-11-01,MP 06: Parkstraat 2 La Filosovia,52.310437,50.104585,58.333332,70.887908,11.0,1.0
4,2022-11-01,MP 07: Naamsestraat 81,50.342093,48.447965,58.920857,70.842471,11.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-11-22,MP 04: His & Hears,56.751950,55.051648,63.782286,75.773857,11.0,22.0
146,2022-11-22,MP 05: Calvariekapel KU Leuven,54.353477,52.350487,60.001033,72.382498,11.0,22.0
147,2022-11-22,MP 06: Parkstraat 2 La Filosovia,54.264537,52.440804,60.319228,72.682259,11.0,22.0
148,2022-11-22,MP 07: Naamsestraat 81,54.103063,52.473697,61.016183,72.828440,11.0,22.0


In [27]:
# check for missing values in each column
print(avg_nov_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64


- December

In [4]:
# combine noise data for December together
noise_dec_combined = pd.concat([naamse35_dec, naamse57_dec, naamse62_dec, calvarie_dec, park_dec, naamse81_dec, kiosk_dec, vrijt_dec, his_dec], axis=0)
noise_dec_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
0,255439,MP 01: Naamsestraat 35 Maxim,01/12/2022 00:00:00.398,56.9,dB(A),55.4,dB(A),59.11,dB(C),71.32,dB(C)
1,255439,MP 01: Naamsestraat 35 Maxim,01/12/2022 00:00:01.398,56.3,dB(A),55.3,dB(A),58.78,dB(C),70.61,dB(C)
2,255439,MP 01: Naamsestraat 35 Maxim,01/12/2022 00:00:02.398,60.1,dB(A),57.0,dB(A),60.73,dB(C),74.49,dB(C)
3,255439,MP 01: Naamsestraat 35 Maxim,01/12/2022 00:00:03.398,59.8,dB(A),55.3,dB(A),60.26,dB(C),72.92,dB(C)
4,255439,MP 01: Naamsestraat 35 Maxim,01/12/2022 00:00:04.398,57.9,dB(A),55.8,dB(A),59.42,dB(C),71.71,dB(C)


In [5]:
# extract the date, month, hour, minute of "result_timestamp"
noise_dec_combined['result_timestamp'] = pd.to_datetime(noise_dec_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


noise_dec_combined['result_date'] = noise_dec_combined['result_timestamp'].dt.date
noise_dec_combined['result_month'] = noise_dec_combined['result_timestamp'].dt.month
noise_dec_combined['result_day'] = noise_dec_combined['result_timestamp'].dt.day
noise_dec_combined['result_hour'] = noise_dec_combined['result_timestamp'].dt.hour
noise_dec_combined['result_minute'] = noise_dec_combined['result_timestamp'].dt.minute

noise_dec_combined.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,result_date,result_month,result_day,result_hour,result_minute
0,255439,MP 01: Naamsestraat 35 Maxim,2022-12-01 00:00:00.398,56.9,dB(A),55.4,dB(A),59.11,dB(C),71.32,dB(C),2022-12-01,12,1,0,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-12-01 00:00:01.398,56.3,dB(A),55.3,dB(A),58.78,dB(C),70.61,dB(C),2022-12-01,12,1,0,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-12-01 00:00:02.398,60.1,dB(A),57.0,dB(A),60.73,dB(C),74.49,dB(C),2022-12-01,12,1,0,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-12-01 00:00:03.398,59.8,dB(A),55.3,dB(A),60.26,dB(C),72.92,dB(C),2022-12-01,12,1,0,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-12-01 00:00:04.398,57.9,dB(A),55.8,dB(A),59.42,dB(C),71.71,dB(C),2022-12-01,12,1,0,0


In [6]:
# aggregate the data by day
avg_dec_combined = noise_dec_combined.groupby(['result_date','description']).mean()
avg_dec_combined = avg_dec_combined.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
avg_dec_combined.drop(columns_to_drop, axis=1, inplace=True)
avg_dec_combined.head(150)

,result_date,description,lamax,laeq,lceq,lcpeak,result_month,result_day
0,2022-12-01,MP 01: Naamsestraat 35 Maxim,56.854533,54.576813,63.007380,75.455894,12.0,1.0
1,2022-12-01,MP 02: Naamsestraat 57 Xior,54.431100,52.331112,60.379675,72.843797,12.0,1.0
2,2022-12-01,MP 03: Naamsestraat 62 Taste,55.961520,53.953686,62.386506,74.578966,12.0,1.0
3,2022-12-01,MP 04: His & Hears,56.366371,54.469186,62.974915,74.927787,12.0,1.0
4,2022-12-01,MP 05: Calvariekapel KU Leuven,54.478829,52.076765,58.858335,71.805250,12.0,1.0
...,...,...,...,...,...,...,...,...
145,2022-12-19,MP 02: Naamsestraat 57 Xior,52.837331,51.007032,63.318394,75.650840,12.0,19.0
146,2022-12-19,MP 03: Naamsestraat 62 Taste,55.085282,53.465877,62.096407,74.016183,12.0,19.0
147,2022-12-19,MP 04: His & Hears,55.972858,54.430023,63.183739,75.188764,12.0,19.0
148,2022-12-19,MP 05: Calvariekapel KU Leuven,53.708619,51.904517,59.452522,71.982617,12.0,19.0


In [7]:
# check for missing values in each column
print(avg_dec_combined.isnull().sum())

result_date     0
description     0
lamax           0
laeq            0
lceq            0
lcpeak          0
result_month    0
result_day      0
dtype: int64
